# Predict tags on StackOverflow with linear models

In this assignment you will learn how to predict tags for posts from [StackOverflow](https://stackoverflow.com). To solve this task you will use multilabel classification approach.

### Libraries

In this task you will need the following libraries:
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [Pandas](https://pandas.pydata.org) — a library providing high-performance, easy-to-use data structures and data analysis tools for the Python
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [NLTK](http://www.nltk.org) — a platform to work with natural language.

### Data

You can find all data required for this assignment into the folder `/data`.

### Text preprocessing

For this assignment you will need to use a list of stop words. It can be downloaded from *nltk*:

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In this task you will deal with a dataset of post titles from StackOverflow. You are provided a split to 3 sets: *train*, *validation* and *test*. All corpora (except for *test*) contain titles of the posts and corresponding tags (100 tags are available). The *test* set doesn't contain answers. Upload the corpora using *pandas* and look at the data:

In [0]:
from ast import literal_eval
import pandas as pd
import numpy as np

Literal_eval package takes care of the preprocessing for the string so that it can be used in python. To know more on literal_eval please see the below documentation <br>
https://kite.com/python/docs/ast.literal_eval

In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [4]:
train = pd.read_csv('/content/gdrive/My Drive/AIML/Projects/StatisticalNLP/train.tsv',delimiter='\t')
train.head()

,title,tags
0,How to draw a stacked dotplot in R?,['r']
1,mysql select all records where a datetime fiel...,"['php', 'mysql']"
2,How to terminate windows phone 8.1 app,['c#']
3,get current time in a specific country via jquery,"['javascript', 'jquery']"
4,Configuring Tomcat to Use SSL,['java']


## Task 1: Create training, testing and validation data from the files given. Use title to be the independent variable and tags to be the dependent variable ( 5 points)

Note: Ensure you apply literal_eval function on the tags column to ensure all the tags are readable in python

In [5]:
val = pd.read_csv('/content/gdrive/My Drive/AIML/Projects/StatisticalNLP/validation.tsv',delimiter='\t')
val.head()

,title,tags
0,Why odbc_exec always fail?,"['php', 'sql']"
1,Access a base classes variable from within a c...,['javascript']
2,"Content-Type ""application/json"" not required i...","['ruby-on-rails', 'ruby']"
3,Sessions in Sinatra: Used to Pass Variable,"['ruby', 'session']"
4,"Getting error - type ""json"" does not exist - i...","['ruby-on-rails', 'ruby', 'json']"


In [6]:
test = pd.read_csv('/content/gdrive/My Drive/AIML/Projects/StatisticalNLP/test.tsv',delimiter='\t')
test.head()

,title
0,Warning: mysql_query() expects parameter 2 to ...
1,get click coordinates from <input type='image'...
2,How to implement cloud storage for media asset...
3,What is catcomplete in jQuery's autocomplete p...
4,Error building Android app with Cordova 3.1 CLI


In [7]:
train[train['tags'].isna()]

,title,tags


In [0]:
train['tags'] = train['tags'].apply(literal_eval)

In [0]:
val['tags'] = val['tags'].apply(literal_eval)

In [10]:
val.head()

,title,tags
0,Why odbc_exec always fail?,"[php, sql]"
1,Access a base classes variable from within a c...,[javascript]
2,"Content-Type ""application/json"" not required i...","[ruby-on-rails, ruby]"
3,Sessions in Sinatra: Used to Pass Variable,"[ruby, session]"
4,"Getting error - type ""json"" does not exist - i...","[ruby-on-rails, ruby, json]"


In [11]:
train.head()

,title,tags
0,How to draw a stacked dotplot in R?,[r]
1,mysql select all records where a datetime fiel...,"[php, mysql]"
2,How to terminate windows phone 8.1 app,[c#]
3,get current time in a specific country via jquery,"[javascript, jquery]"
4,Configuring Tomcat to Use SSL,[java]


## Task 2 (Pre-processing). Implement the function *text_prepare* following the instructions. After that, run the function *test_test_prepare* to test it on tiny cases. (10 points)

One of the most known difficulties when working with natural data is that it's unstructured. For example, if you use it "as is" and extract tokens just by splitting the titles by whitespaces, you will see that there are many "weird" tokens like *3.5*, *?*,  *{}*, etc. To prevent the problems, it's usually useful to prepare the data in a custom way


In [0]:
import re

<h4>Practice Exercise</h4>

In [13]:
title = train['title'][0]
print(title.lower())

how to draw a stacked dotplot in r?


In [14]:
train['title'][23780].lower()

'rotate an image without cropping in opencv in c++'

In [15]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
print(REPLACE_BY_SPACE_RE)

re.compile('[/(){}\\[\\]\\|@,;]')


In [16]:
title = '[parsing text]'
re.sub(REPLACE_BY_SPACE_RE,' ',title)

' parsing text '

In [17]:
print(STOPWORDS)

{"needn't", "shan't", 'how', 'yourself', 'him', 'most', 'over', 'once', "you'd", 'into', 'under', 'between', 'are', "mightn't", "mustn't", 'whom', 'that', 'weren', 'yourselves', 'was', 't', "aren't", 'as', "won't", 'for', 'mightn', 'd', 'and', 'up', 'on', 'if', 'having', 'where', 'it', 'm', 'has', 'ours', 'did', 'by', 'then', 'hadn', 'more', "shouldn't", 'any', 'which', 'while', 'down', "you're", 'they', 'all', 'out', 'can', 'ma', "wouldn't", 'the', 'further', 'in', 'what', 'yours', 'doesn', 'ourselves', 'will', 'shouldn', 'than', 'this', 'during', 'when', 'y', 'again', 'against', 'just', 'were', 'own', 'no', 'too', 'theirs', 'have', 've', 'above', 'to', 'few', "that'll", 'am', 'its', 'nor', 'me', 'do', 'hasn', 'a', 'had', "don't", 'wasn', 'them', 'needn', "isn't", 'been', "should've", 'some', 'himself', "you've", 'both', 'at', 'you', "hasn't", 'through', 'he', 'why', 'such', "didn't", 'same', 'but', 'my', 'from', 'wouldn', 'herself', "weren't", 'an', 'is', 'doing', 'her', 'here', 'its

In [18]:
title = 'yours faithfully'
for i in title.split(' '):
  if not i in STOPWORDS:
    print(i)
print(re.sub(REPLACE_BY_SPACE_RE,'',title))

faithfully
yours faithfully


<h4>Function to delete stopwords from the text</h4>

In [0]:
def deleteStopWordsFromText(stopWords,text):
  filtered_sentence = ''

  for w in text.split(" "):
      if w not in stopWords:
          filtered_sentence = filtered_sentence+' '+w
  return filtered_sentence.strip()

Test one of the example in the test case below

In [0]:
title = 'sql server - any equivalent of excel\'s choose function?'
title = re.sub(REPLACE_BY_SPACE_RE,' ',title)
title = re.sub(BAD_SYMBOLS_RE,'',title)
title = deleteStopWordsFromText(STOPWORDS,title)

<h4>Actual function for text_prepare</h4>

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text): ### The function will take in text and lower case it remove the stopwords, symbols and return it.
    text = text.lower()                            ### Write a code which can change the input text to lowercase.
    text = re.sub(REPLACE_BY_SPACE_RE,' ',text)    ### Write a code which replaces REPLACE_BY_SPACE_RE (above mentioned) symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE,'',text)          ### Write a code which deletes symbols which are in BAD_SYMBOLS_RE (above mentioned) from text
    text = deleteStopWordsFromText(STOPWORDS,text) ### Write a code which deletes stopwords from text
    return re.sub(' +', ' ',text.strip())          ### Delete duplicate spaces in the text

In [0]:
def test_text_prepare():
    examples = ["SQL Server - any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int> * arr?"]
    answers = ["sql server equivalent excels choose function", 
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
        if text_prepare(ex) != ans:
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

Execute the test_text_prepare function

In [23]:
test_text_prepare()

'Basic tests are passed.'

*Note: You should pass the above test to ensure the text preprocessing is done before our analysis*

Now we can preprocess the titles using function *text_prepare* and  making sure that the headers don't have bad symbols:

<h4>Apply for training data</h4>

In [0]:
title1=[]
for trainTitle in train['title']:
  title1.append(text_prepare(trainTitle))

In [0]:
train['title1'] = title1

<h4>Apply for validation data</h4>

In [0]:
val_title1=[]
for valTitle in val['title']:
  val_title1.append(text_prepare(valTitle))

In [0]:
val['title1'] = val_title1

<h4>Apply for test data</h4>

In [0]:
test_title1=[]
for testTitle in test['title']:
  test_title1.append(text_prepare(testTitle))

In [0]:
test['title1'] = test_title1

Print the top 5 elements in x_train

In [30]:
train.head()

,title,tags,title1
0,How to draw a stacked dotplot in R?,[r],draw stacked dotplot r
1,mysql select all records where a datetime fiel...,"[php, mysql]",mysql select records datetime field less speci...
2,How to terminate windows phone 8.1 app,[c#],terminate windows phone 81 app
3,get current time in a specific country via jquery,"[javascript, jquery]",get current time specific country via jquery
4,Configuring Tomcat to Use SSL,[java],configuring tomcat use ssl


In [31]:
train.sample(n=5,random_state=5)

,title,tags,title1
60743,Counting integers with hasNextInt,[java],counting integers hasnextint
33949,Why does Qt use reinterpret_cast rather than s...,"[c++, qt]",qt use reinterpret_cast rather static_cast void
52805,Codebehind OnClick event is not fired after fi...,"[javascript, jquery, asp.net, html5, validation]",codebehind onclick event fired firing jquery e...
11804,Python: using downloaded modules,[python],python using downloaded modules
9229,Randint doesn't always follow uniform distribu...,[python],randint doesnt always follow uniform distribution


In [32]:
val.head()

,title,tags,title1
0,Why odbc_exec always fail?,"[php, sql]",odbc_exec always fail
1,Access a base classes variable from within a c...,[javascript],access base classes variable within child class
2,"Content-Type ""application/json"" not required i...","[ruby-on-rails, ruby]",contenttype application json required rails
3,Sessions in Sinatra: Used to Pass Variable,"[ruby, session]",sessions sinatra used pass variable
4,"Getting error - type ""json"" does not exist - i...","[ruby-on-rails, ruby, json]",getting error type json exist postgresql rake ...


In [33]:
test.head()

,title,title1
0,Warning: mysql_query() expects parameter 2 to ...,warning mysql_query expects parameter 2 resour...
1,get click coordinates from <input type='image'...,get click coordinates input typeimage via java...
2,How to implement cloud storage for media asset...,implement cloud storage media assets zf
3,What is catcomplete in jQuery's autocomplete p...,catcomplete jquerys autocomplete plugin
4,Error building Android app with Cordova 3.1 CLI,error building android app cordova 31 cli


## Task 2 (WordsTagsCount) - Find 3 most popular tags and 3 most popular words in the train data. - 5 points

Note: The words which appear the most are considered as popular in this case!

We are assuming that *tags_counts* and *words_counts* are dictionaries like `{'some_word_or_tag': frequency}`. After applying the sorting procedure, results will be look like this: `[('most_popular_word_or_tag', frequency), ('less_popular_word_or_tag', frequency), ...]`.

eg: 
Tag 1 - 100 Tag 2 - 65 Tag 3 - 250 <br>
after sorting looks like, <br>
Tag 3 - 250 Tag 1 - 100 Tag 2 - 65

<h4>Train Title</h4>

In [34]:
import collections
from nltk import word_tokenize
words_counts = nltk.FreqDist()
titlewords=[]
for i in train['title1']:
  for j in i.split(" "):
    #print(j)
    titlewords.append(j)
print(len(titlewords))
for titleword in titlewords:
  words_counts[titleword]+=1
print(words_counts.most_common(n=100))

565425
[('using', 8278), ('php', 5614), ('java', 5501), ('file', 5055), ('javascript', 4746), ('error', 4388), ('get', 4301), ('c#', 4188), ('python', 4073), ('string', 3652), ('array', 3533), ('data', 3298), ('jquery', 3293), ('value', 3175), ('function', 2896), ('object', 2646), ('class', 2558), ('use', 2422), ('c++', 2312), ('list', 2137), ('method', 2123), ('multiple', 2096), ('text', 1929), ('image', 1849), ('page', 1847), ('rails', 1778), ('form', 1776), ('create', 1757), ('html', 1715), ('type', 1699), ('code', 1664), ('set', 1640), ('working', 1636), ('database', 1631), ('add', 1611), ('convert', 1586), ('mysql', 1572), ('json', 1557), ('c', 1550), ('server', 1543), ('variable', 1542), ('android', 1530), ('values', 1528), ('aspnet', 1506), ('way', 1454), ('application', 1451), ('change', 1441), ('button', 1385), ('find', 1383), ('django', 1373), ('one', 1370), ('without', 1369), ('windows', 1339), ('cannot', 1314), ('table', 1274), ('call', 1261), ('files', 1249), ('ajax', 1248

In [35]:
print(words_counts.most_common(n=5))

[('using', 8278), ('php', 5614), ('java', 5501), ('file', 5055), ('javascript', 4746)]


<h4>Train Tags</h4>

In [36]:
tags_counts = nltk.FreqDist()
tagwords=[]
for k in train['tags']:
  #print(k)
  for m in k:
    tagwords.append(m)
print(len(tagwords))
for tagword in tagwords:
  tags_counts[tagword]+=1
print(tags_counts.most_common(n=100))

194219
[('javascript', 19078), ('c#', 19077), ('java', 18661), ('php', 13907), ('python', 8940), ('jquery', 7510), ('c++', 6469), ('html', 4668), ('objective-c', 4338), ('asp.net', 3939), ('.net', 3872), ('ruby-on-rails', 3344), ('ios', 3256), ('c', 3119), ('mysql', 3092), ('android', 2818), ('ruby', 2326), ('arrays', 2277), ('json', 2026), ('vb.net', 1918), ('iphone', 1909), ('django', 1835), ('css', 1769), ('ajax', 1767), ('r', 1727), ('string', 1573), ('winforms', 1468), ('swift', 1465), ('regex', 1442), ('angularjs', 1353), ('xml', 1347), ('spring', 1346), ('wpf', 1289), ('sql', 1272), ('asp.net-mvc', 1244), ('multithreading', 1118), ('eclipse', 992), ('linq', 964), ('xcode', 900), ('forms', 872), ('html5', 842), ('windows', 838), ('hibernate', 807), ('linux', 793), ('codeigniter', 786), ('node.js', 771), ('swing', 759), ('database', 740), ('list', 693), ('ruby-on-rails-3', 692), ('jsp', 680), ('image', 672), ('entity-framework', 649), ('web-services', 633), ('spring-mvc', 618), ('

In [37]:
print(tags_counts.most_common(n=5))

[('javascript', 19078), ('c#', 19077), ('java', 18661), ('php', 13907), ('python', 8940)]


## Task - 3 Transforming text to a vector (10 points)

Machine Learning algorithms work with numeric data and we cannot use the provided text data "as is". There are many ways to transform text data to numeric vectors. In this task you will try to use two of them.

#### Bag of words

One of the well-known approaches is a *bag-of-words* representation. To create this transformation, follow the steps:
1. Find *N* most popular words in train corpus and numerate them. Now we have a dictionary of the most popular words.
2. For each title in the corpora create a zero vector with the dimension equals to *N*.
3. For each text in the corpora iterate over words which are in the dictionary and increase by 1 the corresponding coordinate.

Let's try to do it for a toy example. Imagine that we have *N* = 4 and the list of the most popular words is 

    ['hi', 'you', 'me', 'are']

Then we need to numerate them, for example, like this: 

    {'hi': 0, 'you': 1, 'me': 2, 'are': 3}

And we have the text, which we want to transform to the vector:

    'hi how are you'

For this text we create a corresponding zero vector 

    [0, 0, 0, 0]
    
And iterate over all words, and if the word is in the dictionary, we increase the value of the corresponding position in the vector:

    'hi':  [1, 0, 0, 0]
    'how': [1, 0, 0, 0] # word 'how' is not in our dictionary
    'are': [1, 0, 0, 1]
    'you': [1, 1, 0, 1]

The resulting vector will be 

    [1, 1, 0, 1]
   
Implement the described encoding in the function *my_bag_of_words* with the size of the dictionary equals to 5000. To find the most common words use train data. You can test your code using the function *test_my_bag_of_words*.

<h4>List of most common (frequent) words in train data</h4>

In [0]:
DICT_SIZE = 5000
freqwords=[]
for i in words_counts.most_common(n=DICT_SIZE):
  count=1
  for item in i:
    if(count==1):  
       freqwords.append(item)
    count=count+1

<h4>Numeration of the most common (frequent) words in train data</h4>

In [0]:
numerated_freq_words={}
numerate_count=0
for freq in freqwords:
  numerated_freq_words[freq]=numerate_count
  numerate_count = numerate_count+1

In [40]:
DICT_SIZE = 5000
WORDS_TO_INDEX =  numerated_freq_words####### YOUR CODE HERE #######
#INDEX_TO_WORDS = ####### YOUR CODE HERE #######
ALL_WORDS = WORDS_TO_INDEX.keys()

text = 'hi how are you'
words_to_index_test =  {'hi': 0, 'you': 1, 'me': 2, 'are': 3}####### YOUR CODE HERE #######

def my_bag_of_words(text, words_to_index, dict_size):
      
      ### Write a function which returns a vector which is a bag-of-words representation of 'text'
      oneText = text
      result_vector = np.zeros(dict_size)
      for i in oneText.split(' '):
        for j,k in zip(words_to_index.keys(),words_to_index.values()):
          if (i==j):
            result_vector.put(k,result_vector[k]+1)
      return result_vector
my_bag_of_words(text,words_to_index_test,4)

array([1., 1., 0., 1.])

In [41]:
res = np.zeros(5)
print(res)
res.put(3,100)
print(res)
str = 'hi'
str1 = 'hi'
print(str==str1)
print(res[3])

[0. 0. 0. 0. 0.]
[  0.   0.   0. 100.   0.]
True
100.0


In [0]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        #print('ex is ',ex)
        #print(my_bag_of_words(ex, words_to_index, 4))
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

Execute the test_text_prepare function <br>
*<u>Note:</u> You should pass the above test to ensure BOW is working correctly!*

In [43]:
test_my_bag_of_words()

'Basic tests are passed.'

Now apply the implemented function to all samples (this might take up to a minute):

In [0]:
X_train = train['title1']
X_val = val['title1']
X_test = test['title1']

In [45]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(100000,)
(30000,)
(20000,)


In [0]:
from scipy import sparse as sp_sparse

In [47]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (100000, 5000)
X_val shape  (30000, 5000)
X_test shape  (20000, 5000)


In [0]:
X1 = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train[0:10]])

In [50]:
for text in X_train[0:10]:
  print(my_bag_of_words(text,WORDS_TO_INDEX,DICT_SIZE))

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[1. 1. 0. ... 0. 0. 0.]


In [51]:
sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text,WORDS_TO_INDEX,DICT_SIZE)) for text in X_train[0:10]])

<10x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 58 stored elements in Compressed Sparse Row format>

As you might notice, we transform the data to sparse representation, to store the useful information efficiently. There are many types: of such representations, however sklearn algorithms can work only with  csr matrix, so we will use this one.<br>
<u>Documentations on sparse matrix:</u> <br>
(https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) 
(https://docs.scipy.org/doc/scipy/reference/sparse.html)




For the 11th row in *X_train_mybag* find how many non-zero elements it has. In this task the answer (variable *non_zero_elements_count*) should be a number, e.g. 20.

In [52]:
array = X_train_mybag[10]
print(array[0])

  (0, 15)	2.0
  (0, 16)	1.0
  (0, 29)	1.0
  (0, 51)	1.0
  (0, 3095)	1.0
  (0, 3326)	1.0
  (0, 4550)	1.0


## Task 4 - TF-IDF (5 points)

The second approach extends the bag-of-words framework by taking into account total frequencies of words in the corpora. It helps to penalize too frequent words and provide better features space. 

Implement function *tfidf_features* using class  from *scikit-learn*. Use *train* corpus to train a vectorizer. Don't forget to take a look into the arguments that you can pass to it. We suggest that you filter out too rare words (occur less than in 5 titles) and too frequent words (occur more than in 90% of the titles). Also, use bigrams along with unigrams in your vocabulary. 

## Write a function which takes x_train, x_val and x_test as input and return the tf-idf features of the same and the vocabulary

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
def tfidf_features(X_train, X_val, X_test):
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = pd.DataFrame(tfidf_vectorizer.fit_transform(X_train).toarray(), columns=tfidf_vectorizer.get_feature_names())
    X_val_tfidf = pd.DataFrame(tfidf_vectorizer.fit_transform(X_val).toarray(), columns=tfidf_vectorizer.get_feature_names())
    X_test_tfidf = pd.DataFrame(tfidf_vectorizer.fit_transform(X_test).toarray(), columns=tfidf_vectorizer.get_feature_names())
    
    return X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vectorizer.vocabulary_

<h4>Session crashes if 40000 training data is used. So, lets use only 35000 training data to create tf-idf matrix</h4>

In [55]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = pd.DataFrame(tfidf_vectorizer.fit_transform(X_train[0:35000]).toarray(), columns=tfidf_vectorizer.get_feature_names())
X_train_tfidf

,00,000,0000,00000000,0000100002,00002efd,000080,00008000,000webhost,0027,...,zoneinfo,zonejs355,zoneoffset,zones,zoom,zoomable,zooming,zopim,zurb,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [56]:
X_val.shape

(30000,)

In [57]:
train.shape

(100000, 3)

In [58]:
val.shape

(30000, 3)

In [59]:
test.shape

(20000, 2)

In [60]:
X_val_tfidf = pd.DataFrame(tfidf_vectorizer.transform(X_val).toarray(), columns=tfidf_vectorizer.get_feature_names())
X_val_tfidf

,00,000,0000,00000000,0000100002,00002efd,000080,00008000,000webhost,0027,...,zoneinfo,zonejs355,zoneoffset,zones,zoom,zoomable,zooming,zopim,zurb,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
X_test.shape

(20000,)

In [62]:
X_test_tfidf = pd.DataFrame(tfidf_vectorizer.transform(X_test).toarray(), columns=tfidf_vectorizer.get_feature_names())
X_test_tfidf

,00,000,0000,00000000,0000100002,00002efd,000080,00008000,000webhost,0027,...,zoneinfo,zonejs355,zoneoffset,zones,zoom,zoomable,zooming,zopim,zurb,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
'''X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train[0:35000], X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}'''

'X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train[0:35000], X_val, X_test)\ntfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}'

In [0]:
tfidf_reversed_vocab = {i:word for word,i in tfidf_vectorizer.vocabulary_.items()}

In [65]:
tfidf_reversed_vocab.values()

dict_values(['draw', 'stacked', 'dotplot', 'mysql', 'select', 'records', 'datetime', 'field', 'less', 'specified', 'value', 'terminate', 'windows', 'phone', '81', 'app', 'get', 'current', 'time', 'specific', 'country', 'via', 'jquery', 'configuring', 'tomcat', 'use', 'ssl', 'awesome', 'nested', 'set', 'plugin', 'add', 'new', 'children', 'tree', 'various', 'levels', 'create', 'map', 'json', 'response', 'ruby', 'rails', 'rspec', 'test', 'method', 'called', 'springboot', 'catalina', 'lifecycle', 'exception', 'import', 'data', 'excel', 'database', 'using', 'php', 'obtaining', 'object', 'javalangclasst', 'parameterized', 'type', 'without', 'constructing', 'class', 'q_uestion', 'ipad', 'selecting', 'text', 'inside', 'input', 'tap', 'jquerys', 'function', 'eclipse', 'mingw', 'lauch', 'program', 'terminated', 'javascript', 'call', 'one', 'prototype', 'another', 'intersection', 'list', 'sets', 'longer', 'able', 'hide', 'keyboard', 'viewwilldisappear', 'ios7', 'fetch', 'key', 'swift', 'change', 

In [66]:
tfidf_reversed_vocab.keys()

dict_keys([4521, 13881, 4473, 9456, 13017, 12048, 3791, 5396, 8145, 13711, 15886, 14770, 16364, 10886, 513, 1183, 5971, 3544, 14973, 13708, 3366, 15993, 7791, 3121, 15085, 15765, 13862, 1660, 9592, 13216, 11064, 818, 9632, 2549, 15221, 15921, 8153, 3410, 8684, 7838, 12452, 12708, 11845, 12694, 14782, 8956, 2244, 13762, 2350, 8192, 5082, 6920, 3651, 5074, 3657, 15813, 10899, 10099, 10051, 7560, 10653, 15332, 16411, 3189, 2633, 11676, 7367, 13038, 14804, 7149, 7118, 14681, 7806, 5842, 4663, 9027, 8078, 11450, 14771, 7607, 2238, 10204, 11532, 1126, 7268, 8263, 13267, 8493, 649, 6468, 7949, 16040, 7356, 5378, 7947, 14409, 2461, 11004, 6408, 14746, 3147, 4815, 8147, 15390, 12140, 8492, 12731, 10317, 1868, 16166, 5114, 5408, 11650, 3164, 10514, 12724, 8625, 14946, 2810, 7527, 2473, 16102, 2472, 8890, 15757, 2022, 9112, 2742, 4848, 11325, 8253, 11768, 6248, 12518, 10810, 8028, 7441, 10287, 7934, 10072, 16004, 12840, 16756, 1669, 1522, 9778, 1519, 13738, 1205, 8761, 12879, 12304, 10187, 6401, 

Print the index of string "C#" in the vocabulary

Since only 35000 training data have been picked up for processing, C# may not be there in that training data

In [0]:
if 'C#' in tfidf_reversed_vocab.values():
  print('Done')
  

## Task 5: Classification (15 points)
MultiLabel classifier

As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose it is convenient to use MultiLabelBinarizer from sklearn. <br>
<u>Documentation:</u> <br>
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
y_train=train['tags']
y_val = val['title1']

In [70]:
print(y_train.shape)

(100000,)


In [71]:
print(y_val.shape)

(30000,)


In [72]:
'''y_train = np.array(y_train).reshape(y_train.shape[0],1)
y_val = np.array(y_val).reshape(y_val.shape[0],1)'''

'y_train = np.array(y_train).reshape(y_train.shape[0],1)\ny_val = np.array(y_val).reshape(y_val.shape[0],1)'

In [73]:
'''print(y_train.shape)
print(y_val.shape)'''

'print(y_train.shape)\nprint(y_val.shape)'

In [74]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) [' ', '#', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] will be ignored
  .format(sorted(unknown, key=str)))


In [75]:
print(y_train.shape)
print(y_val.shape)

(100000, 100)
(30000, 100)


In [76]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [77]:
y_val

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In this task we suggest to use One-vs-Rest approach, which is implemented in [OneVsRestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) class. In this approach *k* classifiers (= number of tags) are trained. As a basic classifier, use [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time, because a number of classifiers to train is large.

**OneVsRest multi-label strategy**

The Multi-label algorithm accepts a binary mask over multiple labels. The result for each prediction will be an array of 0s and 1s marking which class labels apply to each row input sample.

**Logistic Regression & SVM**

OneVsRest strategy can be used for multi-label learning, where a classifier is used to predict multiple labels for instance. LR & SVM supports multi-class, but we are in a multi-label scenario, therefore, we wrap classifiers in the OneVsRestClassifier.

*If you want to learn more about OneVsRest, check out these links:*
- *https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5*
- *https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff*
- *https://medium.com/coinmonks/multi-label-classification-blog-tags-prediction-using-nlp-b0b5ee6686fc*

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

In [0]:
def train_classifier(X_train, y_train):
    # Create and fit LogisticRegression and LinearSVC wraped into OneVsRestClassifier.

    ######################################
    ######### YOUR CODE HERE #############
    ###################################### 
    lr = LogisticRegression()
    #ovr = OneVsRestClassifier(estimator)
    #lsvc  = LinearSVC()
    print(X_train.shape)
    print(y_train.shape)
    lr.fit(X_train,y_train)
    #ovr.fit(X_train,y_train)
    #lsvc.fit(X_train,y_train)
    
    clf = lr
    
    return clf  ### clf is the model

In [80]:
X_train_mybag.shape

(100000, 5000)

In [81]:
X_train_tfidf.shape

(35000, 16757)

In [82]:
y_train.shape

(100000, 100)

In [0]:
y_train = np.argmax(y_train, axis=1)

In [0]:
y_val = np.argmax(y_val, axis=1)

Train the classifiers for different data transformations: *bag-of-words* and *tf-idf*.
classifier_mybag = model for 

In [85]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
classifier_tfidf = train_classifier(X_train_tfidf, y_train[0:35000])

(100000, 5000)
(100000,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(35000, 16757)
(35000,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [86]:
lsvc_bow = LinearSVC()
lsvc_tfidf = LinearSVC()
lsvc_bow.fit(X_train_mybag,y_train)
lsvc_tfidf.fit(X_train_tfidf,y_train[0:35000])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [87]:
from sklearn.naive_bayes import MultinomialNB
ovc_bow = OneVsRestClassifier(lsvc_bow)
ovc_tfidf = OneVsRestClassifier(lsvc_bow)
ovc_bow.fit(X_train_mybag,y_train)
ovc_tfidf.fit(X_train_tfidf,y_train[0:35000])

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None)

Now you can create predictions for the data. You will need two types of predictions: labels and scores.

In [0]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

In [0]:
y_val_predicted_labels_lvc_mybag = lsvc_bow.predict(X_val_mybag)
y_val_predicted_scores_lvc_mybag = lsvc_bow.decision_function(X_val_mybag)

y_val_predicted_labels_lvc_tfidf = lsvc_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_lvc_tfidf = lsvc_tfidf.decision_function(X_val_tfidf)

In [0]:
y_val_predicted_labels_ovc_mybag = ovc_bow.predict(X_val_mybag)
y_val_predicted_scores_ovc_mybag = ovc_bow.decision_function(X_val_mybag)

y_val_predicted_labels_ovc_tfidf = ovc_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_ovc_tfidf = ovc_tfidf.decision_function(X_val_tfidf)

Now, we would need to compare the results of different predictions, e.g. to see whether TF-IDF transformation helps or to try different regularization techniques in logistic regression. For all these experiments, we need to setup evaluation procedure. 

## Evaluation (10 points)

To evaluate the results we will use several classification metrics:
 - [Accuracy](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
 - [F1-score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)
 - [Area under ROC-curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
 - [Area under precision-recall curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score) 
 
Make sure you are familiar with all of them. If you want a refresher, you can click the link to their documentation

## Import the necessary libraries for the above metrics

In [0]:
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,precision_recall_curve

Define the function *print_evaluation_scores* which takes y_val and predicted as input calculates and prints the following output:
 - *accuracy*
 - *F1-score - Average = 'weighted'* 
 - *Precision - Average = 'macro'*

In [0]:
def print_evaluation_scores(y_val, predicted):
  print('accuracy score is ',accuracy_score(y_val,predicted))
  print('f1 score is ',f1_score(y_val,predicted,average=None))
  #print('roc_auc_score is ',roc_auc_score(y_val,predicted,average=None))
  #print('precision_recall_curve is ',precision_recall_curve(y_val,predicted))
    


In [93]:
print('Bag-of-words')
print_evaluation_scores(y_val, y_val_predicted_labels_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

Bag-of-words
accuracy score is  0.024966666666666668
f1 score is  [0.01597444 0.         0.         0.         0.         0.
 0.         0.         0.         0.05383767 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.02772964 0.
 0.         0.         0.         0.         0.         0.
 0.        ]
Tfidf
accuracy score is  0.0146
f1 score is  [0.01574803 0.         0.         0.         0.         0.
 0.         0.         0.         0.03175653 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [94]:
print('Bag-of-words')
print_evaluation_scores(y_val, y_val_predicted_labels_lvc_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_lvc_tfidf)

Bag-of-words
accuracy score is  0.028433333333333335
f1 score is  [0.01851852 0.         0.         0.         0.         0.
 0.         0.         0.         0.06089926 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.03157714 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
Tfidf
accuracy score is  0.025866666666666666
f1 score is  [0.02166985 0.         0.         0.         0.         0.
 0.         0.         0.         0.05570033 0.         0.
 0.         0.         0.         0.         0.    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [95]:
print('Bag-of-words')
print_evaluation_scores(y_val, y_val_predicted_labels_ovc_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_ovc_tfidf)

Bag-of-words
accuracy score is  0.028433333333333335
f1 score is  [0.01851852 0.         0.         0.         0.         0.
 0.         0.         0.         0.06089926 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.03157714 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
Tfidf
accuracy score is  0.025866666666666666
f1 score is  [0.02166985 0.         0.         0.         0.         0.
 0.         0.         0.         0.05570033 0.         0.
 0.         0.         0.         0.         0.    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


You might also want to plot some form of the [ROC curve](http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc) for the case of multi-label classification. The input parameters for the roc curve are:
 - true labels
 - decision functions scores
 - number of classes

Import the roc_auc function from the metrics.py file provided

## Task 4 (MultilabelClassification) - Optional 
** Once we have the evaluation set up, we suggest that you experiment a bit with training your classifiers. We will use *F1-score weighted* as an evaluation metric. Our recommendation:
- compare the quality of the bag-of-words and TF-IDF approaches and chose one of them.
- for the chosen one, try *L1* and *L2*-regularization techniques in Logistic Regression with different coefficients (e.g. C equal to 0.1, 1, 10, 100).



In [0]:
######################################
######### YOUR CODE HERE #############
######################################

When you are happy with the quality, create predictions for *test* set.